# Multus

![](https://github.com/k8snetworkplumbingwg/multus-cni/raw/master/docs/images/Multus.png)

![](https://github.com/k8snetworkplumbingwg/multus-cni/raw/master/docs/images/multus-pod-image.svg)

Quelle: https://github.com/k8snetworkplumbingwg/multus-cni
- - -

Multus CNI ist ein Container Network Interface (CNI)-Plugin für Kubernetes, das das Anfügen mehrerer Netzwerkschnittstellen an Pods ermöglicht.


In [ ]:
# microk8s Variante
! sudo microk8s enable community
! sudo microk8s enable multus
# Multus Variante
! # kubectl apply -f https://raw.githubusercontent.com/k8snetworkplumbingwg/multus-cni/master/deployments/multus-daemonset-thick.yml

- - -

## Netzwerk mit Bridge-Plugin

Mit dem Bridge-Plugin werden alle Container (auf demselben Host) an eine Bridge (virtueller Switch) angeschlossen, die sich im Namespace des Hostnetzwerks befindet. 

Die Container erhalten ein Ende des Veth-Paares (Virtual Ethernet Device), während das andere Ende mit der Bridge verbunden ist.

Der Bridge selbst wird ebenfalls eine IP-Adresse zugewiesen werden, wodurch sie zu einem Gateway für die Container wird. 

Quelle: https://www.cni.dev/plugins/current/main/bridge/

In [ ]:
%%bash
cat <<EOF | kubectl apply -f -
---
apiVersion: "k8s.cni.cncf.io/v1"
kind: NetworkAttachmentDefinition
metadata:
  name: bridge-network
spec:
  config: '{
    "cniVersion": "0.3.1",
    "name": "mynet",
    "type": "bridge",
    "bridge": "mynet0",
    "isDefaultGateway": true,
    "forceAddress": false,
    "ipMasq": true,
    "hairpinMode": true,
    "ipam": {
        "type": "host-local",
        "subnet": "10.10.0.0/16"
        }
    }'
EOF

Zum Testen erstellen wir einen Pod mit zwei Netzwerk Adapter (ohne loopback).

In [ ]:
%%bash
cat <<EOF | kubectl create -f -
apiVersion: v1
kind: Pod
metadata:
  name: multus-autoshop
  annotations:
    k8s.v1.cni.cncf.io/networks: bridge-network
spec:
  containers:
  - name: multus-autoshop
    image: registry.gitlab.com/ch-mc-b/autoshop/shop:2.0.0
EOF

In [ ]:
! kubectl exec -it multus-autoshop -- ip a
! kubectl exec -it multus-autoshop -- route

Der Container ist nun über das Default Pod Netzwerk und über das 10.10.0.0/16 Netzwerk erreichbar

In [ ]:
! curl http://10.10.0.3:8080

- - -

### Aufräumen

In [ ]:
! kubectl delete pod multus-autoshop
! kubectl delete network-attachment-definitions bridge-network

- - -

### Quellen:

* [Auto Shop](https://gitlab.com/ch-mc-b/autoshop/shop)
* [CNI - Die Container-Netzwerkschnittstelle](https://www.cni.dev/)